# Extract BARRA/ERA5

In [1]:
import sys
import os
import glob
import xarray as xr
sys.path.append('/g/data/qz03/wp3/code/git-repo')
from utils import *

import dask
import tempfile
from dask.diagnostics import ProgressBar
from dask.distributed import Client, LocalCluster
import warnings
import logging
warnings.filterwarnings('ignore') 
logging.getLogger("distributed.worker.memory").setLevel(logging.ERROR)
logging.getLogger('flox').setLevel(logging.WARNING)

### Extract BARRA (faster without dask)

In [2]:
extracted_data_save_dir = '/g/data/qz03/wp3/data/reanalysis/'
time_step = '1hr' #10min 1hr 3hr day fx mon
nation_domain = 'indonesia'

for target_var in ['tas', 'uas', 'vas', 'rsds', 'sfcWind', 'sfcWindmax', 'pr', 'va100m', 'ua100m']:
    print(target_var)
    for year in list(range(1979,2025)):
      barra_extract(target_var, extracted_data_save_dir, time_step, nation_domain, year)

tas
uas
vas
rsds
sfcWind
sfcWindmax
pr
va100m
ua100m


### Extract ERA5 (faster with dask)

In [3]:
dask.config.set({'array.chunk-size': "256 MiB",'array.slicing.split_large_chunks': True, 'distributed.comm.timeouts.connect': '120s', 'distributed.comm.timeouts.tcp': '120s', 'distributed.comm.retry.count': 10, 'distributed.scheduler.allowed-failures': 20, "distributed.scheduler.worker-saturation": 1.1})
client = Client(n_workers=12, threads_per_worker=1, local_directory = tempfile.mkdtemp(), memory_limit = "63000mb")

In [4]:
extracted_data_save_dir = '/g/data/qz03/wp3/data/reanalysis/'
nation_domain = 'indonesia'

for target_var in ['10u', '10v', 'ssrd', 'tp', '2t', '100u', '100v']:
    print(target_var)
    for year in list(range(1979,2025)):
      era5_extract(target_var, extracted_data_save_dir, nation_domain, year)  

10u
10v
ssrd
tp
2t
100u
100v


In [5]:
client.close()